<a href="https://colab.research.google.com/github/eliasyanez/TC1001S.100-202211/blob/main/Mini_Reto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Configuración Inicial y Librerías


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install librosa pandas pydub

import os
import librosa
import numpy as np
import pandas as pd
from pydub import AudioSegment
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
from sklearn.naive_bayes import GaussianNB

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Conversión de Audios de Entrenamiento de m4a a wav

In [ ]:
# Ruta a la carpeta donde están los archivos .m4a
audio_dir = '/content/drive/My Drive/ITD/Int Artificial Avanzada/Tareas/Tarea equipo/Train/'

In [ ]:
# Convertir los archivos a .wav
for file_name in os.listdir(audio_dir):
    if file_name.endswith('.m4a'):
        file_path = os.path.join(audio_dir, file_name)
        audio = AudioSegment.from_file(file_path, format="m4a")
        wav_file_path = file_path.replace('.m4a', '.wav')
        audio.export(wav_file_path, format="wav")

3. Extracción de Características y Creación del Archivo CSV

In [ ]:
def extract_features(file_path, n_mfcc=13):
    """
    Extrae MFCCs de un archivo de audio.

    :param file_path: Ruta al archivo de audio
    :param n_mfcc: Número de coeficientes MFCC a extraer
    :return: Vector de características MFCCs
    """
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
        mfccs_mean = np.mean(mfccs.T, axis=0)  # Promedio de los MFCCs a través del tiempo
        return mfccs_mean
    except Exception as e:
        print(f"Error al procesar {file_path}: {e}")
        return None

In [ ]:
# Inicializar listas para almacenar las características y etiquetas
features = []
labels = []

# Procesar los audios en la carpeta
for file_name in os.listdir(audio_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(audio_dir, file_name)
        mfccs = extract_features(file_path)
        if mfccs is not None:
            features.append(mfccs)
            # Etiquetar según el nombre del archivo
            if "Sobrio" in file_name:
                labels.append(0)  # Sobrio
            elif "Ebrio" in file_name:
                labels.append(1)  # Ebrio

# Convertir a DataFrame para un manejo más sencillo
df = pd.DataFrame(features)
df['label'] = labels

# Guardar las características y etiquetas en un archivo CSV para uso posterior
df.to_csv('/content/drive/My Drive/ITD/Int Artificial Avanzada/Tareas/Tarea equipo/Train/audio_features_labels.csv', index=False)

4. Entrenamiento del Modelo de Regresión Logística


In [ ]:
# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/drive/My Drive/ITD/Int Artificial Avanzada/Tareas/Tarea equipo/Train/audio_features_labels.csv')

# Separar las características (X) y las etiquetas (y)
X = data.iloc[:, :-1]  # Todas las columnas excepto la última
y = data['label']  # Última columna es la etiqueta

# Estandarizar las características (normalizar)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Entrenar el modelo de Regresión Logística
model = LogisticRegression(random_state=42)
model.fit(X, y)

# Guardar el modelo y el escalador entrenados
joblib.dump(model, 'audio_classifier_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("Modelo de regresión logística entrenado y guardado exitosamente.")

Modelo de regresión logística entrenado y guardado exitosamente.


5. Conversión de Audios de Prueba de m4a a wav


In [ ]:
# Ruta a la carpeta donde están los archivos de prueba .m4a
test_audio_dir = '/content/drive/My Drive/ITD/Int Artificial Avanzada/Tareas/Tarea equipo/Test/'

In [ ]:
# Convertir los archivos de prueba a .wav
for file_name in os.listdir(test_audio_dir):
    if file_name.endswith('.m4a'):
        file_path = os.path.join(test_audio_dir, file_name)
        audio = AudioSegment.from_file(file_path, format="m4a")
        wav_file_path = file_path.replace('.m4a', '.wav')
        audio.export(wav_file_path, format="wav")

6. Extracción de Características de los Audios de Prueba


In [ ]:
# Inicializar listas para almacenar las características y etiquetas de prueba
test_features = []
test_labels = []

# Procesar los audios de prueba
for file_name in os.listdir(test_audio_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(test_audio_dir, file_name)
        mfccs = extract_features(file_path)
        if mfccs is not None:
            test_features.append(mfccs)
            # Etiquetar según el nombre del archivo
            if "Test sobrio" in file_name:
                test_labels.append(0)  # Sobrio
            elif "Test ebrio" in file_name:
                test_labels.append(1)  # Ebrio

# Convertir la lista de características a un array numpy
X_test_new = np.array(test_features)
y_test_new = np.array(test_labels)

# Estandarizar las características
X_test_new = scaler.transform(X_test_new)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


7. Evaluación del Modelo de Regresión Logística con Datos de Prueba


In [ ]:
# Realizar predicciones con el modelo entrenado
y_pred_new = model.predict(X_test_new)

# Evaluar las predicciones
accuracy = accuracy_score(y_test_new, y_pred_new)
conf_matrix = confusion_matrix(y_test_new, y_pred_new)
class_report = classification_report(y_test_new, y_pred_new)

print(f"Exactitud del modelo en pruebas: {accuracy * 100:.2f}%")
print("Matriz de confusión:")
print(conf_matrix)
print("Informe de clasificación:")
print(class_report)

Exactitud del modelo en pruebas: 45.00%
Matriz de confusión:
[[7 3]
 [8 2]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.47      0.70      0.56        10
           1       0.40      0.20      0.27        10

    accuracy                           0.45        20
   macro avg       0.43      0.45      0.41        20
weighted avg       0.43      0.45      0.41        20



8. Ajuste del Umbral de Decisión


In [ ]:
# Obtener las probabilidades de predicción
y_pred_proba = model.predict_proba(X_test_new)[:, 1]

# Ajustar el umbral (ejemplo: 0.3 en lugar de 0.5)
threshold = 0.3
y_pred_custom_threshold = (y_pred_proba >= threshold).astype(int)

# Evaluar las predicciones con el nuevo umbral
accuracy_custom = accuracy_score(y_test_new, y_pred_custom_threshold)
conf_matrix_custom = confusion_matrix(y_test_new, y_pred_custom_threshold)
class_report_custom = classification_report(y_test_new, y_pred_custom_threshold)

print(f"Exactitud del modelo con umbral ajustado en pruebas: {accuracy_custom * 100:.2f}%")
print("Matriz de confusión:")
print(conf_matrix_custom)
print("Informe de clasificación:")
print(class_report_custom)

Exactitud del modelo con umbral ajustado en pruebas: 45.00%
Matriz de confusión:
[[6 4]
 [7 3]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.46      0.60      0.52        10
           1       0.43      0.30      0.35        10

    accuracy                           0.45        20
   macro avg       0.45      0.45      0.44        20
weighted avg       0.45      0.45      0.44        20



9. Evaluación con Naive Bayes


In [ ]:
# Crear el modelo de Naive Bayes
nb_model = GaussianNB()

# Entrenar el modelo con los datos de entrenamiento
nb_model.fit(X, y)

# Realizar predicciones con los datos de prueba
y_pred_nb = nb_model.predict(X_test_new)

# Evaluar las predicciones
accuracy_nb = accuracy_score(y_test_new, y_pred_nb)
conf_matrix_nb = confusion_matrix(y_test_new, y_pred_nb)
class_report_nb = classification_report(y_test_new, y_pred_nb)

print(f"Exactitud del modelo Naive Bayes en pruebas: {accuracy_nb * 100:.2f}%")
print("Matriz de confusión:")
print(conf_matrix_nb)
print("Informe de clasificación:")
print(class_report_nb)

Exactitud del modelo Naive Bayes en pruebas: 25.00%
Matriz de confusión:
[[2 8]
 [7 3]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.22      0.20      0.21        10
           1       0.27      0.30      0.29        10

    accuracy                           0.25        20
   macro avg       0.25      0.25      0.25        20
weighted avg       0.25      0.25      0.25        20



10. Inversión de Predicciones


In [ ]:
# Invertir las predicciones del modelo
y_pred_inverted = 1 - y_pred_nb  # Cambia 0 a 1 y 1 a 0

# Evaluar las predicciones invertidas
accuracy_inverted = accuracy_score(y_test_new, y_pred_inverted)
conf_matrix_inverted = confusion_matrix(y_test_new, y_pred_inverted)
class_report_inverted = classification_report(y_test_new, y_pred_inverted)

print(f"Exactitud del modelo Naive Bayes invertido en pruebas: {accuracy_inverted * 100:.2f}%")
print("Matriz de confusión (invertida):")
print(conf_matrix_inverted)
print("Informe de clasificación (invertido):")
print(class_report_inverted)

Exactitud del modelo Naive Bayes invertido en pruebas: 75.00%
Matriz de confusión (invertida):
[[8 2]
 [3 7]]
Informe de clasificación (invertido):
              precision    recall  f1-score   support

           0       0.73      0.80      0.76        10
           1       0.78      0.70      0.74        10

    accuracy                           0.75        20
   macro avg       0.75      0.75      0.75        20
weighted avg       0.75      0.75      0.75        20



In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import joblib

# Guardar el modelo entrenado
joblib.dump(model, 'audio_classifier_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
import streamlit as st
import numpy as np
import librosa
import joblib
from sklearn.preprocessing import StandardScaler

code = """
import streamlit as st
import numpy as np
import librosa
import joblib
from sklearn.preprocessing import StandardScaler

# Cargar el modelo y el escalador
model = joblib.load('audio_classifier_model.pkl')
scaler = joblib.load('scaler.pkl')

# Título de la app
st.title('Clasificación de Audio: Sobrio vs Ebrio')

# Subir archivo de audio
audio_file = st.file_uploader("Sube un archivo de audio", type=['wav', 'm4a'])

if audio_file is not None:
    # Procesar el archivo de audio
    st.audio(audio_file, format='audio/wav')
    audio_data, sample_rate = librosa.load(audio_file, sr=None)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
    mfccs_mean = np.mean(mfccs.T, axis=0)

    # Normalizar las características
    mfccs_scaled = scaler.transform([mfccs_mean])

    # Realizar la predicción
    prediction = model.predict(mfccs_scaled)
    prob = model.predict_proba(mfccs_scaled)[0]

    # Mostrar el resultado
    if prediction[0] == 0:
        st.write(f"**Predicción: Sobrio** (Probabilidad: {prob[0]*100:.2f}%)")
    else:
        st.write(f"**Predicción: Ebrio** (Probabilidad: {prob[1]*100:.2f}%)")
"""

# Guardar el código en un archivo llamado 'app.py' en Google Drive
with open('/content/drive/My Drive/ITD/Int Artificial Avanzada/Tareas/Tarea equipo/app.py', 'w') as f:
    f.write(code)

In [ ]:
!streamlit run app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
